In [1]:
import pandas as pd
import numpy as np
import glob, os, sys
from hmmlearn import hmm
osp = sys.platform
# import matplotlib.pyplot as plt
# %matplotlib inline

from utilities import PlotGame, LoadData, make_video
from helpers import id_player, id_position, get_player_trajectory
# ---------------------------------------------------------
%load_ext autoreload
%autoreload 2

In [2]:
gameid='0021500463'

# directories
# CHANGE HERE
main_dir = '../'
game_dir = main_dir+'data/'
# court_path = data_dir + 'nba_court_T.png'

Data = LoadData(main_dir, game_dir)
data = Data.load_game(gameid)
event_number, moment_number = 0, 0


In [3]:
# get events and number of events
event_df = pd.DataFrame(data['events'])
n_events = event_df.shape[0]
# get numebr of moments
moments = event_df.iloc[event_number,:]['moments']
n_moments = len(moments)

print(n_moments, event_df.shape)
event_df.head()

150 (231, 8)


,end_time_left,home,moments,orig_events,playbyplay,quarter,start_time_left,visitor
0,702.31,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351428029, 708.28, 12.78, None, [[-1,...",[0],GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,708.28,"{'abbreviation': 'TOR', 'players': [{'playerid..."
1,686.28,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351428029, 708.28, 12.78, None, [[-1,...",[1],GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,708.28,"{'abbreviation': 'TOR', 'players': [{'playerid..."
2,668.42,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351444029, 692.25, 12.21, None, [[-1,...","[2, 3]",GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,692.25,"{'abbreviation': 'TOR', 'players': [{'playerid..."
3,665.71,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351463029, 673.22, 21.44, None, [[-1,...","[4, 5]",GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,673.22,"{'abbreviation': 'TOR', 'players': [{'playerid..."
4,646.64,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351473027, 668.0, 22.9, None, [[-1, -...",[6],GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,668.00,"{'abbreviation': 'TOR', 'players': [{'playerid..."


In [4]:
data.keys()

dict_keys(['gamedate', 'gameid', 'events'])

In [5]:
event_df.loc[0,:]

end_time_left                                                 702.31
home               {'abbreviation': 'CHI', 'players': [{'playerid...
moments            [[1, 1451351428029, 708.28, 12.78, None, [[-1,...
orig_events                                                      [0]
playbyplay               GAME_ID  EVENTNUM  EVENTMSGTYPE  EVENTMS...
quarter                                                            1
start_time_left                                               708.28
visitor            {'abbreviation': 'TOR', 'players': [{'playerid...
Name: 0, dtype: object

In [6]:
event_df.loc[0,:]['moments'][0][5]

[[-1, -1, 18.38063, 14.07976, 8.56325],
 [1610612741, 2200, 11.15334, 21.35529, 0.0],
 [1610612741, 201959, 16.79035, 20.55978, 0.0],
 [1610612741, 201565, 9.12233, 39.32051, 0.0],
 [1610612741, 202710, 21.15543, 32.71616, 0.0],
 [1610612741, 202703, 8.40459, 11.67492, 0.0],
 [1610612761, 2449, 24.18381, 44.21187, 0.0],
 [1610612761, 201960, 1.06327, 2.54971, 0.0],
 [1610612761, 200768, 7.28146, 48.40417, 0.0],
 [1610612761, 201942, 18.1243, 14.25539, 0.0],
 [1610612761, 202687, 10.82794, 26.70275, 0.0]]

In [7]:
np.array(event_df.loc[0,:]['moments'][0][5])

array([[ -1.00000000e+00,  -1.00000000e+00,   1.83806300e+01,
          1.40797600e+01,   8.56325000e+00],
       [  1.61061274e+09,   2.20000000e+03,   1.11533400e+01,
          2.13552900e+01,   0.00000000e+00],
       [  1.61061274e+09,   2.01959000e+05,   1.67903500e+01,
          2.05597800e+01,   0.00000000e+00],
       [  1.61061274e+09,   2.01565000e+05,   9.12233000e+00,
          3.93205100e+01,   0.00000000e+00],
       [  1.61061274e+09,   2.02710000e+05,   2.11554300e+01,
          3.27161600e+01,   0.00000000e+00],
       [  1.61061274e+09,   2.02703000e+05,   8.40459000e+00,
          1.16749200e+01,   0.00000000e+00],
       [  1.61061276e+09,   2.44900000e+03,   2.41838100e+01,
          4.42118700e+01,   0.00000000e+00],
       [  1.61061276e+09,   2.01960000e+05,   1.06327000e+00,
          2.54971000e+00,   0.00000000e+00],
       [  1.61061276e+09,   2.00768000e+05,   7.28146000e+00,
          4.84041700e+01,   0.00000000e+00],
       [  1.61061276e+09,   2.0194200

In [8]:
# loop through each events and grab the same id
# first find the max moments
event_df['moments_length'] = event_df['moments'].apply(lambda m: len(m))
event_df.head(2)

,end_time_left,home,moments,orig_events,playbyplay,quarter,start_time_left,visitor,moments_length
0,702.31,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351428029, 708.28, 12.78, None, [[-1,...",[0],GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,708.28,"{'abbreviation': 'TOR', 'players': [{'playerid...",150
1,686.28,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351428029, 708.28, 12.78, None, [[-1,...",[1],GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,708.28,"{'abbreviation': 'TOR', 'players': [{'playerid...",550


In [9]:
event_df.iloc[0, :].home['players']

[{'firstname': 'Bobby',
  'jersey': '5',
  'lastname': 'Portis',
  'playerid': 1626171,
  'position': 'F'},
 {'firstname': 'Derrick',
  'jersey': '1',
  'lastname': 'Rose',
  'playerid': 201565,
  'position': 'G'},
 {'firstname': 'Cameron',
  'jersey': '41',
  'lastname': 'Bairstow',
  'playerid': 203946,
  'position': 'F-C'},
 {'firstname': 'Nikola',
  'jersey': '44',
  'lastname': 'Mirotic',
  'playerid': 202703,
  'position': 'F'},
 {'firstname': 'Jimmy',
  'jersey': '21',
  'lastname': 'Butler',
  'playerid': 202710,
  'position': 'G-F'},
 {'firstname': 'Taj',
  'jersey': '22',
  'lastname': 'Gibson',
  'playerid': 201959,
  'position': 'F'},
 {'firstname': "E'Twaun",
  'jersey': '55',
  'lastname': 'Moore',
  'playerid': 202734,
  'position': 'G'},
 {'firstname': 'Tony',
  'jersey': '20',
  'lastname': 'Snell',
  'playerid': 203503,
  'position': 'F'},
 {'firstname': 'Pau',
  'jersey': '16',
  'lastname': 'Gasol',
  'playerid': 2200,
  'position': 'C-F'},
 {'firstname': 'Kirk',
  

In [10]:
# get all the player_id and player_name mapping
player_id_mapping = id_player(event_df)

In [11]:
player_id_mapping

{2200: 'Pau Gasol',
 2449: 'Luis Scola',
 2550: 'Kirk Hinrich',
 200768: 'Kyle Lowry',
 201166: 'Aaron Brooks',
 201565: 'Derrick Rose',
 201942: 'DeMar DeRozan',
 201949: 'James Johnson',
 201959: 'Taj Gibson',
 201960: 'DeMarre Carroll',
 202335: 'Patrick Patterson',
 202685: 'Jonas Valanciunas',
 202687: 'Bismack Biyombo',
 202703: 'Nikola Mirotic',
 202709: 'Cory Joseph',
 202710: 'Jimmy Butler',
 202734: "E'Twaun Moore",
 203082: 'Terrence Ross',
 203461: 'Anthony Bennett',
 203503: 'Tony Snell',
 203512: 'Lucas Nogueira',
 203926: 'Doug McDermott',
 203946: 'Cameron Bairstow',
 1626153: 'Delon Wright',
 1626171: 'Bobby Portis',
 1626245: 'Cristiano Felicio'}

In [12]:
# get position mapping
# get all the player_id and player_name mapping
position_id_mapping = id_position(event_df)

In [13]:
position_id_mapping

{2200: ['C-F'],
 2449: ['F'],
 2550: ['G'],
 200768: ['G'],
 201166: ['G'],
 201565: ['G'],
 201942: ['G'],
 201949: ['F'],
 201959: ['F'],
 201960: ['F'],
 202335: ['F'],
 202685: ['C'],
 202687: ['C-F'],
 202703: ['F'],
 202709: ['G'],
 202710: ['G-F'],
 202734: ['G'],
 203082: ['F-G'],
 203461: ['F'],
 203503: ['F'],
 203512: ['C'],
 203926: ['F'],
 203946: ['F-C'],
 1626153: ['G'],
 1626171: ['F'],
 1626245: ['F-C']}

In [14]:
set(np.concatenate(list(position_id_mapping.values())))

{'C', 'C-F', 'F', 'F-C', 'F-G', 'G', 'G-F'}

In [15]:
# first unique player ids for each event
def get_player_id(moments):
    return list(set([i[1] for m in moments for i in m[5]]))
event_df['players_id'] = event_df['moments'].apply(get_player_id)
# also obtain all the ids
all_ids = list(set([j for i in event_df['players_id'].values for j in i]))
# remove the -1 in player id
all_ids = [i for i in all_ids if i >= 0]
event_df.head()

,end_time_left,home,moments,orig_events,playbyplay,quarter,start_time_left,visitor,moments_length,players_id
0,702.31,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351428029, 708.28, 12.78, None, [[-1,...",[0],GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,708.28,"{'abbreviation': 'TOR', 'players': [{'playerid...",150,"[200768, 201959, 201960, 202703, 2449, 202710,..."
1,686.28,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351428029, 708.28, 12.78, None, [[-1,...",[1],GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,708.28,"{'abbreviation': 'TOR', 'players': [{'playerid...",550,"[200768, 201959, 201960, 202703, 2449, 202710,..."
2,668.42,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351444029, 692.25, 12.21, None, [[-1,...","[2, 3]",GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,692.25,"{'abbreviation': 'TOR', 'players': [{'playerid...",596,"[200768, 201959, 201960, 202703, 2449, 202710,..."
3,665.71,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351463029, 673.22, 21.44, None, [[-1,...","[4, 5]",GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,673.22,"{'abbreviation': 'TOR', 'players': [{'playerid...",314,"[200768, 201959, 201960, 202703, 2449, 202710,..."
4,646.64,"{'abbreviation': 'CHI', 'players': [{'playerid...","[[1, 1451351473027, 668.0, 22.9, None, [[-1, -...",[6],GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,668.00,"{'abbreviation': 'TOR', 'players': [{'playerid...",627,"[200768, 201959, 201960, 202703, 2449, 202710,..."


In [ ]:
all_ids

In [ ]:
# first get the trajectory of a single player
pid = 2200
    
pid_track = pd.DataFrame(event_df['moments'].apply(lambda m: get_player_trajectory(m, pid)))
pid_track = pid_track[pid_track.moments.apply(lambda m: len(m) != 0)]
pid_track.reset_index(inplace=True, drop=True)
pid_track['length'] = pid_track.moments.apply(lambda m: len(m))

In [ ]:
pid_track.shape

In [ ]:
pid_track.head()

In [ ]:
pid_track.moments.values.shape

In [ ]:
train = np.array([j for i in pid_track.moments.values for j in i])
train.shape

In [ ]:
length = pid_track.length.values

In [ ]:

# Viterbi is set by default as the 'algorithm' optional parameter.
model = hmm.GMMHMM(n_components=5, n_mix=5)#, covariance_type="diag", n_iter=100) 
model.fit(train, lengths=length)

In [ ]:
state_sequence = model.predict(train, lengths=length)
# prob_next_step = model.transmat_[state_sequence[-1], :]

In [ ]:
prob_next_step*100

In [ ]:
state_sequence.shape

In [ ]:
state_sequence[:150]

---

### Next: try using the predefined position as indexing mechanism for the imitation learning

Questions:

  - Is the latent structure learning trained for each individual agent or their sequences are put together?